## SET UP & DATA WRANGLING

In [1]:
import pandas as pd
import numpy as np # funciones matemáticas

## EDA Libraries 
import matplotlib.pyplot as plt
import seaborn as sns

## Settings
pd.set_option('display.max_columns', None) # display all columns
import warnings
warnings.filterwarnings('ignore') # ignore warnings

In [2]:
# pip install --upgrade openpyxl

In [3]:
data = pd.read_excel("typeform 15-09-2024.xlsx")

In [4]:
df = data.copy()

In [5]:
df.columns = df.columns.str.lower().str.replace(" ","_").str.replace(".","_").str.replace(":","")

In [6]:
df.shape

(1239, 24)

In [7]:
df.sample(10)

,#,¿cómo_te_gusta_que_te_llamemos?,"para_empezar,_aceptas_la_política_de_privacidad_y_que_te_mandemos_chicha_de_la_buena?","{{field01g9cmc9sx965qwjp9reb0q2bj}},_escribe_tu_mejor_correo","escribe_tu_teléfono_(*con_prefijo,_si_no_no_te_podremos_contactar*)_para_poder_pasarte_la_información_de_nuestra_sesión_juntas❤️",¿en_qué_franja_horaria_vives?,"-nuestra_terapia_grupal_dura_*16_semanas_*\n\n-los_horarios_son_de_*1200_-_1400_pm_o_de_1900_a_2100_pm_*(franja_horaria_españa)\n\n-tiene_un_precio_desde*_64€_por_sesiones_de_2_horas__*pero_no_se_paga_por_sesión,_sino_que_te_comprometes_al_proceso_entero_\n\n_*esto_implica_tanto_una_inversión_económica,_como_de_tiempo_*__si_esto_no_encaja_en_tus_posibilidades,_te_pedimos,_con_amor,_que_reconsideres_la_reserva__","cuéntame,_¿qué_no_está_funcionando_actualmente_en_tu_vida?",¿_qué_cambios_te_gustaría_ver_después_de_pasar_por_nuestra_terapia?,"{{field01g9cmc9sx965qwjp9reb0q2bj}},_¿por_qué_crees_que_conecta_contigo,_nuestra_terapia_grupal,_puede_ayudarte?_","-nuestra_terapia_grupal_dura_*16_semanas_*\n\n-cada_semana_de_*1900-2100_pm_o_cada_semana_de_1200-1400_pm_*\n-se_paga_por_*todo_el_proceso*_en_cuotas_de_1/3/6_pagos_mensuales_\n\nesto_implica_una_inversión_económica_y_de_tiempo__si_esto_no_encaja_en_tus_posibilidades,_te_pedimos,_con_amor,_que_reconsideres_la_reserva__\n\n¿estás_interesada_en_continuar?",sí!,"no,_solo_por_la_tarde!",por_las_tardes_a_partir_de_las_3_00pm,por_las_tardes_a_partir_de_las_5_00pm,por_las_tardes_a_partir_de_las_6_00pm,por_las_tardes_a_partir_de_las_7_00pm,"tu_tiempo_es_oro_y_el_nuestro_también,_reserva_esta_sesión_solo_*si_estás_comprometida_a_asistir__*\n\n*-esta_sesión_es_una_sesión_enfocada_a_ver_tu_caso_para_entrar_a_la_terapia_grupal_*_\n\nsin_tu_compromiso,_no_puede_haber_avance__¿trato_hecho?",response_type,start_date_(utc),stage_date_(utc),submit_date_(utc),network_id,tags
483,hz2p8hpjpnkkvdoqhz2p8anejjk74vzf,Bren,Acepto! yes!,bcamachoperez@pucpr.edu,1.939247e+10,NaN,NaN,Me he quedado enganchada con un casi algo me g...,Recuperar mi confianza. Para mi era un vínculo...,Porque mi terapeuta me ha ayudado en todas las...,"Sí, estoy dispuesta a invertir en mí y tengo d...",NaN,NaN,NaN,NaN,NaN,NaN,Sí!! Estoy dispuesta a invertirme en mí y comp...,completed,2024-02-10 00:04:24,NaN,2024-02-10 00:17:21,a90f07be4e,NaN
277,65dlx0x1dj5bnywyvpmd9n65dlx0xz2s,Maca,Acepto! yes!,caleromacarena93@gmail.com,3.461962e+10,Europa,NaN,No me quiero y soy muy autodestructiva conmigo...,Tener mas y mejor autoestima,"Soy una persona muy abierta, y me ayuda mucho ...","Sí, estoy dispuesta a invertir en mí y tengo d...",NaN,NaN,NaN,NaN,NaN,NaN,Sí!! Estoy dispuesta a invertirme en mí y comp...,completed,2024-05-01 12:35:40,NaN,2024-05-01 12:38:21,5b61c3b662,NaN
1022,74sz8n7y0ilbjdzddi74sz8pedr9boux,Natalia,Acepto! yes!,sisuka00@hotmail.com,NaN,NaN,NaN,No encuentro pareja,Ser capaz de conectar con alguien,Porque para poder conectar con alguien primero...,"Sí, estoy dispuesta a invertir en mí y tengo d...",NaN,NaN,NaN,NaN,NaN,NaN,"No, estoy comprometida ni puedo invertir prefi...",completed,2023-02-09 22:17:41,NaN,2023-02-09 22:19:56,b050896985,NaN
671,5a6p4ro15ztnwt7q5a6p4a8mgq1x7aaa,Mari,Acepto! yes!,mariaranda32@gmail.com,3.466355e+10,NaN,NaN,"Algo anda mal dentro de mi, llevo un par de añ...","Sentirme bién conmigo, y que lo que hago lo ha...","No lo sé, a lo mejor por sentirme escuchada","Sí, estoy dispuesta a invertir en mí y tengo d...",NaN,NaN,NaN,NaN,NaN,NaN,Sí!! Estoy dispuesta a invertirme en mí y comp...,completed,2023-11-02 12:05:17,NaN,2023-11-02 12:13:24,817c604b36,NaN
253,4sgx52jzjq8vxsvdr4sgx5lyisujgxs0,Sadith,Acepto! yes!,sadithquirozl@gmail.com,5.195518e+10,Latino America/USA,No se ajusta a mis posibilidades 🙏🏽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,completed,2024-05-14 06:13:27,NaN,2024-05-14 06:14:34,3272781825,NaN
1025,n7q0593xss1oe1gln7q05f0jg6ylu398,Linda,NaN,linda.ordonez@asterchile.cl,NaN,NaN,NaN,La aceptación,Vivir tranquila con mis decisiones,Porque 

In [8]:
df.columns

Index(['#', '¿cómo_te_gusta_que_te_llamemos?',
       'para_empezar,_aceptas_la_política_de_privacidad_y_que_te_mandemos_chicha_de_la_buena?',
       '{{field01g9cmc9sx965qwjp9reb0q2bj}},_escribe_tu_mejor_correo',
       'escribe_tu_teléfono_(*con_prefijo,_si_no_no_te_podremos_contactar*)_para_poder_pasarte_la_información_de_nuestra_sesión_juntas❤️',
       '¿en_qué_franja_horaria_vives?',
       '-nuestra_terapia_grupal_dura_*16_semanas_*\n\n-los_horarios_son_de_*1200_-_1400_pm_o_de_1900_a_2100_pm_*(franja_horaria_españa)\n\n-tiene_un_precio_desde*_64€_por_sesiones_de_2_horas__*pero_no_se_paga_por_sesión,_sino_que_te_comprometes_al_proceso_entero_\n\n_*esto_implica_tanto_una_inversión_económica,_como_de_tiempo_*__si_esto_no_encaja_en_tus_posibilidades,_te_pedimos,_con_amor,_que_reconsideres_la_reserva__',
       'cuéntame,_¿qué_no_está_funcionando_actualmente_en_tu_vida?',
       '¿_qué_cambios_te_gustaría_ver_después_de_pasar_por_nuestra_terapia?',
       '{{field01g9cmc9sx965qwjp9re

In [9]:
df.drop(columns=["#","sí!","no,_solo_por_la_tarde!","por_las_tardes_a_partir_de_las_3_00pm","por_las_tardes_a_partir_de_las_5_00pm","por_las_tardes_a_partir_de_las_6_00pm","por_las_tardes_a_partir_de_las_7_00pm", "stage_date_(utc)", "tags"], inplace=True)

In [10]:
df = df.rename(columns={"¿cómo_te_gusta_que_te_llamemos?":"nombre",
                   "para_empezar,_aceptas_la_política_de_privacidad_y_que_te_mandemos_chicha_de_la_buena?":"politica_privacidad",
                   "{{field01g9cmc9sx965qwjp9reb0q2bj}},_escribe_tu_mejor_correo":"email",
                   "escribe_tu_teléfono_(*con_prefijo,_si_no_no_te_podremos_contactar*)_para_poder_pasarte_la_información_de_nuestra_sesión_juntas❤️":"telefono",
                   "¿en_qué_franja_horaria_vives?":"zona_horaria",
                   "-nuestra_terapia_grupal_dura_*16_semanas_*\n\n-los_horarios_son_de_*1200_-_1400_pm_o_de_1900_a_2100_pm_*(franja_horaria_españa)\n\n-tiene_un_precio_desde*_64€_por_sesiones_de_2_horas__*pero_no_se_paga_por_sesión,_sino_que_te_comprometes_al_proceso_entero_\n\n_*esto_implica_tanto_una_inversión_económica,_como_de_tiempo_*__si_esto_no_encaja_en_tus_posibilidades,_te_pedimos,_con_amor,_que_reconsideres_la_reserva__":"condiciones_terapia",
                   "cuéntame,_¿qué_no_está_funcionando_actualmente_en_tu_vida?":"problema_principal",
                   "¿_qué_cambios_te_gustaría_ver_después_de_pasar_por_nuestra_terapia?":"cambios_deseados",
                   "{{field01g9cmc9sx965qwjp9reb0q2bj}},_¿por_qué_crees_que_conecta_contigo,_nuestra_terapia_grupal,_puede_ayudarte?_":"busqueda_en_terapia",
                   "-nuestra_terapia_grupal_dura_*16_semanas_*\n\n-cada_semana_de_*1900-2100_pm_o_cada_semana_de_1200-1400_pm_*\n-se_paga_por_*todo_el_proceso*_en_cuotas_de_1/3/6_pagos_mensuales_\n\nesto_implica_una_inversión_económica_y_de_tiempo__si_esto_no_encaja_en_tus_posibilidades,_te_pedimos,_con_amor,_que_reconsideres_la_reserva__\n\n¿estás_interesada_en_continuar?":"interes_reserva",
                   "tu_tiempo_es_oro_y_el_nuestro_también,_reserva_esta_sesión_solo_*si_estás_comprometida_a_asistir__*\n\n*-esta_sesión_es_una_sesión_enfocada_a_ver_tu_caso_para_entrar_a_la_terapia_grupal_*_\n\nsin_tu_compromiso,_no_puede_haber_avance__¿trato_hecho?":"interes_sesion_personal",
                   "start_date_(utc)":"start_date",
                   "submit_date_(utc)":"submit_date"
                  })

In [11]:
df.sample(5)

,nombre,politica_privacidad,email,telefono,zona_horaria,condiciones_terapia,problema_principal,cambios_deseados,busqueda_en_terapia,interes_reserva,interes_sesion_personal,response_type,start_date,submit_date,network_id
327,Zaymy,Acepto! yes!,zaymypaulino@gmail.com,4.917772e+11,Europa,NaN,"Me siento sola, procrastinando mucho y sufrien...",Me gustaría entenderme más y poder conectar co...,Nunca he asistido a terapia grupal y me causa ...,"Sí, estoy dispuesta a invertir en mí y tengo d...",Sí!! Estoy dispuesta a invertirme en mí y comp...,completed,2024-04-01 22:41:59,2024-04-01 22:46:02,6acb82028b
104,Marcela,Acepto! yes!,Yanez.psp@gmail.com,5.697205e+10,Latino America/USA,No se ajusta a mis posibilidades 🙏🏽,NaN,NaN,NaN,NaN,NaN,completed,2024-07-17 02:12:44,2024-07-17 02:14:23,829e320e19
894,Montse,Acepto! yes!,montse.hoy@hotmail.com,NaN,NaN,NaN,Yo misma,"Más contenta e ilusionada, feliz",Porqué he oído tus audios y me han parecido y ...,"Sí, estoy dispuesta a invertir en mí y tengo d...",Sí!! Estoy dispuesta a invertirme en mí y comp...,completed,2023-05-09 17:25:02,2023-05-09 17:27:59,4e20b97f07
980,Àngela,Acepto! yes!,angelaggels@gmail.com,NaN,NaN,NaN,Mi forma de gestionar ciertas situaciones soci...,Paz y calma en mi interior. Aprender a gestion...,"No me cierro a nada, y amo el podcast, me sien...","Sí, estoy dispuesta a invertir en mí y tengo d...",Sí!! Estoy dispuesta a invertirme en mí y comp...,completed,2023-02-28 08:23:37,2023-02-28 08:27:00,c09dcdc8b4
206,Joi,Acepto! yes!,kaicomplejo@gmail.com,5.411762e+11,Latino America/USA,No se ajusta a mis posibilidades 🙏🏽,NaN,NaN,NaN,NaN,NaN,completed,2024-06-04 12:22:18,2024-06-04 12:23:23,b7df496c37


In [12]:
df.dtypes

nombre                             object
politica_privacidad                object
email                              object
telefono                          float64
zona_horaria                       object
condiciones_terapia                object
problema_principal                 object
cambios_deseados                   object
busqueda_en_terapia                object
interes_reserva                    object
interes_sesion_personal            object
response_type                      object
start_date                 datetime64[ns]
submit_date                datetime64[ns]
network_id                         object
dtype: object

### Leyenda de columnas:

**nombre**: nombre de pila

**politica_privacidad**: acepta políticas de privacidad y consentimiento publicitario

**email**: correo electrónico

**telefono**: telefono con extensión

**zona_horaria**: 3 zonas contempladas (Europa, Latam/USA, Asia/Australia)

**condiciones_terapia**: la usuaria puede asumir condiciones y precio de terapia grupal

**problema_principal**: qué no funciona en sus vidas para buscar terapia grupal

**cambios_deseados**: qué buscan cambiar gracias a la terapia grupal

**busqueda_en_terapia**: cómo creen que la terapia grupañ les ayudará

**interes_reserva**: si están interesadas en reservar plaza

**interes_sesion_personal**: si están interesadas en realizar una sesión personalizada sobre su caso en específico de cara a entrar en la terapia grupal

**start_date**: momento en que comienzan el typeform

**submit_date**: momento en que finalizan el typeform

**network_id**: id específico de su entrada

## STANDARIZATION & NULLS

### Cleaning functions

In [13]:
# replace and fillna

def replace_fillna(df, column_name, replace_dict=None, fill_value=None):
    if replace_dict:
        df[column_name] = df[column_name].replace(replace_dict)
    if fill_value is not None:
        df[column_name] = df[column_name].fillna(fill_value)
    return df

In [14]:
# format clean telephone

def format_phone(phone):
    if pd.isna(phone):
        return phone
    else:
        phone_str = str(phone)
        phone_str = phone_str.split(".")[0]
        return f"+{phone_str[:2]} {phone_str[2:5]} {phone_str[5:7]} {phone_str[7:9]} {phone_str[9:]}"

In [15]:
# Obtainin phone code number

def country_code(value, skip_value="Sin completar"):
    if value == skip_value:
        return value
    else:
        return str(value).split(" ")[0]
    
def apply_country_code(df, column_name, new_column_name, skip_value="Sin completar"):
    df[new_column_name] = df[column_name].apply(lambda x: country_code(x, skip_value))
    return df[new_column_name]

In [16]:
# Obtainin country through phone code

def country(code, skip_value="Sin completar"):
    if code == skip_value:
        return "Desconocido"
    else:
        return country_by_code.get(code)

### Política de Privacidad

In [17]:
df["politica_privacidad"].value_counts()

politica_privacidad
Acepto! yes!    1069
Name: count, dtype: int64

In [18]:
df = replace_fillna(df, "politica_privacidad", replace_dict={"Acepto! yes!":"Yes"}, fill_value="No")

In [19]:
df["politica_privacidad"].value_counts()

politica_privacidad
Yes    1069
No      170
Name: count, dtype: int64

#### Teléfono

In [20]:
df["telefono"]

0       5.233226e+11
1       5.198916e+10
2       3.934979e+11
3       5.492604e+12
4       5.699234e+10
            ...     
1234             NaN
1235             NaN
1236             NaN
1237             NaN
1238             NaN
Name: telefono, Length: 1239, dtype: float64

In [21]:
df["telefono"] = df["telefono"].apply(format_phone)

In [22]:
df["telefono"].isna().sum()

372

In [23]:
df["telefono"] = df["telefono"].fillna("Sin completar")

In [24]:
df["telefono"].sample(10)

411      +34 607 08 64 28
591      +34 633 36 06 32
78       +34 646 13 25 46
157      +34 686 90 84 84
1113        Sin completar
587     +44 746 38 80 707
1157        Sin completar
51       +34 649 17 44 68
1054        Sin completar
2       +39 349 79 31 372
Name: telefono, dtype: object

### Zona horaria

In [25]:
df["zona_horaria"].value_counts()

zona_horaria
Europa                231
Latino America/USA    124
Asia/Australia          6
Name: count, dtype: int64

In [26]:
df = replace_fillna(df, "zona_horaria", replace_dict={"Europa":"Europa", "Latino America/USA":"Latam/USA", "Asia/Australia":"Asia/Australia"}, fill_value="Sin completar")

In [27]:
df["zona_horaria"].value_counts()

zona_horaria
Sin completar     878
Europa            231
Latam/USA         124
Asia/Australia      6
Name: count, dtype: int64

### Condiciones terapia

In [28]:
df["condiciones_terapia"].value_counts(dropna=False)

condiciones_terapia
NaN                                    1115
No se ajusta a mis posibilidades 🙏🏽     106
Sí se ajusta a mis posibilidades ❤️      18
Name: count, dtype: int64

In [29]:
df = replace_fillna(df, "condiciones_terapia", replace_dict={"No se ajusta a mis posibilidades 🙏🏽":"No (Latam)", "Sí se ajusta a mis posibilidades ❤️":"Yes (Latam)"}, fill_value="Yes")

In [30]:
df["condiciones_terapia"].value_counts(dropna=False)

condiciones_terapia
Yes            1115
No (Latam)      106
Yes (Latam)      18
Name: count, dtype: int64

#### Problema principal - Cambios deseados - Búsqueda en terapia

In [31]:
columnas_nan = ["problema_principal", "cambios_deseados", "busqueda_en_terapia"]

In [32]:
df[columnas_nan] = df[columnas_nan].fillna("Sin completar")

### Interés reserva

In [33]:
df["interes_reserva"].value_counts(dropna=False)

interes_reserva
Sí, estoy dispuesta a invertir en mí y tengo disponibilidad semanal.                                        439
Sí, estoy dispuesta a invertir en mí y tengo disponibilidad semanal, aunque me cuadraría pagar a plazos.    331
No, ahora mismo no estoy dispuesta a invertir en terapia.                                                   258
NaN                                                                                                         211
Name: count, dtype: int64

In [34]:
df = replace_fillna(df, "interes_reserva", replace_dict={"Sí, estoy dispuesta a invertir en mí y tengo disponibilidad semanal.":"Yes", 
                                                         "Sí, estoy dispuesta a invertir en mí y tengo disponibilidad semanal, aunque me cuadraría pagar a plazos.":"Yes (Pago plazos)",
                                                         "No, ahora mismo no estoy dispuesta a invertir en terapia.":"No"
                                                        }, 
                                                         fill_value="Sin completar")

In [35]:
df["interes_reserva"].value_counts(dropna=False)

interes_reserva
Yes                  439
Yes (Pago plazos)    331
No                   258
Sin completar        211
Name: count, dtype: int64

#### Interés Sesión personalizada

In [36]:
df["interes_sesion_personal"].value_counts(dropna=False)

interes_sesion_personal
Sí!! Estoy dispuesta a invertirme en mí y comprometida a asistir.    820
NaN                                                                  265
No, estoy comprometida ni puedo invertir prefiero no seguir          154
Name: count, dtype: int64

In [37]:
df = replace_fillna(df, "interes_sesion_personal", replace_dict={"Sí!! Estoy dispuesta a invertirme en mí y comprometida a asistir.":"Yes", 
                                                         "No, estoy comprometida ni puedo invertir prefiero no seguir":"No"
                                                        }, 
                                                         fill_value="Sin completar")

In [38]:
df["interes_sesion_personal"].value_counts(dropna=False)

interes_sesion_personal
Yes              820
Sin completar    265
No               154
Name: count, dtype: int64

In [39]:
df.sample(10)

,nombre,politica_privacidad,email,telefono,zona_horaria,condiciones_terapia,problema_principal,cambios_deseados,busqueda_en_terapia,interes_reserva,interes_sesion_personal,response_type,start_date,submit_date,network_id
928,Cami,Yes,camilakohn13@hmail.com,Sin completar,Sin completar,Yes,Me cuesta tener confianza conmigo misma y soltar.,"Me gustaría recuperar la confianza, empatizar ...",Me interesa que sea grupal y me gusta escucharlas,Yes (Pago plazos),Yes,completed,2023-04-11 11:51:36,2023-04-11 11:56:10,1996b5f172
302,x,Yes,notengo@nontengo.com,+34 600 00 00 00,Europa,Yes,lkjlkj,ljlkjkl,lkjlkj,Yes,Yes,completed,2024-04-18 18:38:57,2024-04-18 18:39:54,55a66153e2
22,Noeli,Yes,noe_7_33@hotmail.com,+34 680 62 73 69,Europa,Yes,"Mis relaciones con los chicos , tuve una relac...",Un aumento de mi autoestima que creo que es la...,Creo que puede ser una bonita forma de avanzar,No,Sin completar,completed,2024-09-02 20:55:10,2024-09-02 20:58:06,e5fec7880d
1083,Laura,Yes,laurapruwe@gmail.com,Sin completar,Sin completar,Yes,Mi compromiso con mi crecimiento personal y há...,"Comprometida con mis hábitos, mayor conocimien...",Es creada específicamente para mujeres,Yes (Pago plazos),Sin completar,completed,2023-01-22 17:04:39,2023-01-22 17:09:06,e4c28f1c0b
413,Mayo,Yes,marjoriesanchez1508@gmail.com,+56 926 34 90 73,Sin completar,Yes,Mi vida en general,Tener más autoestima,Por los videos que vi creo que me ayudaría,No,Yes,completed,2024-02-28 06:17:29,2024-02-28 06:20:18,83f2ac944b
1157,Marija,No,masha.sufik@gmail.com,Sin completar,Sin completar,Yes,Sin completar,Sin completar,Sin completar,Sin completar,Sin completar,completed,2022-11-11 09:38:08,2022-11-11 09:39:07,1d7d32ea53
131,Gema,Yes,gemis_85@hotmail.com,+34 650 73 74 84,Europa,Yes,No lo sé,Saber quién soy,Para poder conocer a otras personas,No,Sin completar,completed,2024-07-02 16:23:30,2024-07-02 16:23:45,fa5eba557f
999,Dúnia,No,duniafeesparza@gmail.com,Sin completar,Sin completar,Yes,Mi vida personal es un caos,Necesito dejar a mi pareja y seguir con mi vid...,Me hace ver las cosas desde la perspectiva que...,Yes,Yes,completed,2023-02-18 23:27:18,2023-02-18 23:29:37,24151bce92
679,Sandra,Yes,bizamatani@gmail.com,+54 261 24 09 466,Sin completar,Yes,Altibajos emocionales,Alcanzar más serenidad conmigo misma,Porque estoy segura que el trabajo con otros m...,No,No,completed,2023-10-25 11:10:50,2023-10-25 11:14:38,67e3212385
221,Carina,Yes,flycaryfer@gmail.com,+84 393 44 95 83,Asia/Australia,Yes,Motivación y relaciones sociales,Mejor estado de animo,Para escuchar y reflexionar,Yes (Pago plazos),Yes,completed,2024-05-28 15:34:41,2024-05-28 15:38:47,1a48b8b07a


### *New column*: país

In [40]:
df_code = apply_country_code(df, "telefono", "codigo_tlf")

In [41]:
df_code = df_code.reset_index(drop=True)

In [42]:
df_code = pd.DataFrame(df_code)

In [43]:
df_code.sample(5)

,codigo_tlf
1157,Sin completar
231,+34
1143,Sin completar
518,+34
433,+34


In [44]:
country_by_code = {
"+34": "España",
"+52": "México",
"+56": "Chile",
"+54": "Argentina",
"+57": "Colombia",
"+59": "Bolivia",
"+50": "Guatemala",
"+51": "Perú",
"+39": "Italia",
"+41": "Suiza",
"+58": "Venezuela",
"+17": "Estados Unidos",
"+19": "Canadá",
"+61": "Australia",
"+12": "Estados Unidos",
"+18": "Estados Unidos",
"+33": "Francia",
"+44": "Reino Unido",
"+49": "Alemania",
"+96": "Bahréin",
"+15": "Estados Unidos",
"+55": "Brasil",
"+13": "Estados Unidos",
"+16": "Estados Unidos",
"+97": "Emiratos Árabes Unidos",
"+32": "Bélgica",
"+35": "Gibraltar",
"+14": "Estados Unidos",
"+38": "Yugoslavia",
"+86": "China",
"+37": "Lituania",
"+84": "Vietnam",
"+36": "Hungría",
"+46": "Suecia",
"+31": "Países Bajos",
"+48": "Polonia",
"+47": "Noruega",
"+64": "Nueva Zelanda"
}

In [45]:
df_code["pais"] = df_code["codigo_tlf"].apply(country)

In [46]:
df_code.head(5)

,codigo_tlf,pais
0,+52,México
1,+51,Perú
2,+39,Italia
3,+54,Argentina
4,+56,Chile


In [47]:
df_columns = df.columns.to_list()

In [48]:
df_columns.index("zona_horaria")

4

In [49]:
df.insert(5, "pais", df_code["pais"])

In [52]:
df.drop(columns=["codigo_tlf"], inplace=True)

In [53]:
df.sample(5)

,nombre,politica_privacidad,email,telefono,zona_horaria,pais,condiciones_terapia,problema_principal,cambios_deseados,busqueda_en_terapia,interes_reserva,interes_sesion_personal,response_type,start_date,submit_date,network_id
362,Cariño,Yes,miguel@hortelanos.net,+34 635 72 54 21,Sin completar,España,Yes,Tengo miedos que han aparecido en mi vida y an...,Que desaparezcan dichos miedos.,No estoy seguro.,Yes,Yes,completed,2024-03-18 10:41:37,2024-03-18 10:44:23,b54de4edb7
724,Pili,Yes,pcheda@uoc.edu,+34 625 05 73 54,Sin completar,España,Yes,Relación de pareja,"Conseguir una relación de pareja no monógama, ...",No lo sé,No,No,completed,2023-09-24 06:06:12,2023-09-24 06:10:31,04830b3c44
266,pene,Yes,pene@gmail.com,+34 666 66 66 66,Europa,España,Yes,pene,puta,caca,Yes,Yes,completed,2024-05-05 14:01:34,2024-05-05 14:03:52,6ca15cf200
94,Laura,Yes,lauralorenafeijoo@gmail.com,+54 116 00 06 555,Latam/USA,Argentina,No (Latam),Sin completar,Sin completar,Sin completar,Sin completar,Sin completar,completed,2024-07-26 05:53:01,2024-07-26 05:54:11,ad99ff0bff
793,Mel,Yes,mel.castruitac@gmail.com,+52 558 61 62 511,Sin completar,México,Yes,"Mi relación de pareja, mi seguridad y autoestima","Amor propio, dejar de sentir apego , dejar de ...",Por qué los podcasts me han ayudado y me eh se...,Yes (Pago plazos),Yes,completed,2023-08-01 14:39:13,2023-08-01 14:44:21,9c2bb39880


## CALENDLY MERGE: SESSIONS DONE

### Set up

In [54]:
data2 = pd.read_excel("Calendly 15 sp 2024.xlsx")

In [55]:
df2 = data2.copy()

In [62]:
df2.columns = df2.columns.str.lower().str.replace(" ","_").str.replace(".","_").str.replace(":","")

In [63]:
df2.sample(1)

,user_name,team,invitee_name,invitee_first_name,invitee_last_name,invitee_email,invitee_time_zone,invitee_accepted_marketing_emails,text_reminder_number,event_type_name,start_date_&_time,end_date_&_time,location,event_created_date_&_time,canceled,canceled_by,cancellation_reason,question_1,response_1,question_2,response_2,question_3,response_3,question_4,response_4,question_5,response_5,question_6,response_6,question_7,response_7,question_8,response_8,utm_campaign,utm_source,utm_medium,utm_term,utm_content,salesforce_uuid,event_price,payment_currency,guest_email(s),invitee_reconfirmed,marked_as_no-show,meeting_notes,group,user_email,event_uuid,invitee_uuid,invitee_scheduled_by,scheduling_method
556,Eyas Psicología,NaN,Ainhoa Egaña hernandez,Ainhoa,Egaña hernandez,ainhoi_eh@hotmail.com,Europa Central,NaN,NaN,Sesión Conecta Contigo,2024-05-16 16:30:00,2024-05-16 17:15:00,Zoom URL: https://us05web.zoom.us/j/8799213570...,2024-05-13 13:40:00,False,NaN,NaN,Tu teléfono para poder mandarte el link de la ...,669107748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,wadasanchez@gmail.com,1af110e3-2461-4527-bf11-b13d58cd1059,73a8c55d-547a-48e0-8e7f-f6a794282af2,Eyas Psicología,NaN


In [61]:
df2.shape

(629, 51)

In [64]:
calendly = df2[["invitee_email", "start_date_&_time", "end_date_&_time"]]

In [65]:
calendly

,invitee_email,start_date_&_time,end_date_&_time
0,wadasanchez@gmail.com,2021-06-02 10:00:00,2021-06-02 10:45:00
1,eganie@gmail.com,2021-07-15 20:00:00,2021-07-15 20:45:00
2,kbperona@gmail.com,2021-09-29 20:00:00,2021-09-29 20:45:00
3,esti_kamatxo@hotmail.com,2022-01-03 09:00:00,2022-01-03 10:00:00
4,jessi17246@hotmail.es,2022-01-03 20:00:00,2022-01-03 21:00:00
...,...,...,...
624,ainatml@gmail.com,2024-09-03 18:00:00,2024-09-03 19:00:00
625,ainatml@gmail.com,2024-09-05 13:00:00,2024-09-05 14:00:00
626,ana.osuna1980@gmail.com,2024-09-10 15:00:00,2024-09-10 16:00:00
627,machezgui2@gmail.com,2024-09-05 10:00:00,2024-09-05 11:00:00


In [66]:
df["sesion_completada"] = df["email"].isin(calendly["invitee_email"]).map({True: "Yes", False: "No"})

In [68]:
df["sesion_completada"].value_counts()

sesion_completada
No     700
Yes    539
Name: count, dtype: int64

### Sessions done

In [69]:
sesiones_si = df[df["sesion_completada"] == "Yes"]

In [71]:
sesiones_si["pais"].value_counts()

pais
España                    281
Desconocido               149
México                     16
Argentina                  14
Colombia                   11
Chile                      10
Bolivia                     8
Perú                        6
Guatemala                   5
Estados Unidos              5
Suiza                       4
Brasil                      3
Australia                   3
Bahréin                     3
Italia                      3
Canadá                      2
Gibraltar                   2
Reino Unido                 2
Emiratos Árabes Unidos      2
Venezuela                   2
Polonia                     1
Alemania                    1
Suecia                      1
Hungría                     1
Lituania                    1
China                       1
Francia                     1
Bélgica                     1
Name: count, dtype: int64

In [72]:
sesiones_no = df[df["sesion_completada"] == "No"]

In [73]:
sesiones_no["pais"].value_counts()

pais
España            248
Desconocido       223
Chile              44
México             39
Argentina          35
Estados Unidos     23
Colombia           20
Guatemala          12
Italia              9
Bolivia             9
Perú                8
Venezuela           5
Canadá              4
Francia             4
Alemania            3
Suiza               3
Australia           3
Reino Unido         2
Nueva Zelanda       1
Vietnam             1
Países Bajos        1
Yugoslavia          1
Bélgica             1
Noruega             1
Name: count, dtype: int64

## EXPORT CLEAN CSV TO EDA

In [75]:
df.to_csv("clean_typeform.csv", index=False)